In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os

# Laden der Excel-Daten
data_excel_file_path = 'station_data_analysis.xlsx'

if not os.path.exists(data_excel_file_path):
    raise FileNotFoundError(f"Die Datei {data_excel_file_path} wurde nicht gefunden.")

try:
    station_data = pd.read_excel(data_excel_file_path, sheet_name='Station Status')
    station_info = pd.read_excel(data_excel_file_path, sheet_name='Station Information')
    vehicle_types = pd.read_excel(data_excel_file_path, sheet_name='Vehicle Types')
    soll_werte = pd.read_excel(data_excel_file_path, sheet_name='Soll Werte')
except Exception as e:
    raise ValueError(f"Fehler beim Laden der Excel-Datei: {e}")

# Definition der Lubus IDs
lubu_ids = [
    "958219cd-8009-45b6-8a04-1cc4ae763307",
    "714932fd-4836-4a2a-9a19-07a6d694274c",
    "be796eb3-49af-494c-b3f0-dc1f0132eb7d",
    "c070e40c-98f6-45d6-9543-90ceceef63af",
    "ca41b63e-9046-442f-880e-d0e9186e507a",
    "36fe6a30-d53e-4aa8-863d-e21e80fd1d0b",
    "7046d131-47fd-40bd-ba63-f52590d2ee8f",
    "2263b695-db35-4bbf-b70a-032920fadf3f"
]

# Hilfsstrukturen erstellen
id_to_name = pd.Series(station_info['name'].values, index=station_info['station_id']).to_dict()
vehicle_type_mapping = vehicle_types.set_index('vehicle_type_id')['name'].to_dict()

# Funktion zur Extraktion der Fahrzeuganzahl
def extract_vehicle_count(station_entries, vehicle_type_name):
    counts = []
    for entry in station_entries:
        count = 0
        if isinstance(entry, dict) and 'vehicle_data' in entry:
            vehicle_data = entry['vehicle_data']
            for vehicle in vehicle_data:
                if vehicle_type_mapping.get(vehicle['vehicle_type_id']) == vehicle_type_name:
                    count = vehicle['count']
                    break
        counts.append(count)
    return counts

# Funktion zur Visualisierung der Stationsdaten
def visualize_station_data(station_id_requested, bike_type='all', compare_with_soll=False):
    plt.figure(figsize=(12, 8))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%Y %H:%M'))
    plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
    plt.gcf().autofmt_xdate()

    if bike_type == 'bike':
        vehicle_type_name = 'bike'
        soll_column = 'bike'
    elif bike_type == 'pedelec':
        vehicle_type_name = 'pedelec'
        soll_column = 'pedelec'
    elif bike_type == 'cargopedelec':
        vehicle_type_name = 'cargopedelec'
        soll_column = 'cargopedelec'
    else:
        vehicle_type_name = 'all'
        soll_column = 'all'
        
    if station_id_requested.lower() == "alle":
        for station_id in lubu_ids:
            if station_id in station_data['station_id'].values:
                filtered_data = station_data[station_data['station_id'] == station_id].to_dict('records')
                
                if vehicle_type_name == 'all':
                    counts = [entry['num_bikes_available'] for entry in filtered_data]
                    soll_value = soll_werte[soll_werte['station_id'] == station_id][['bike', 'pedelec', 'cargopedelec']].sum(axis=1).values[0]
                else:
                    counts = extract_vehicle_count(filtered_data, vehicle_type_name)
                    soll_value = soll_werte[soll_werte['station_id'] == station_id][soll_column].values[0]
                
                plt.plot([entry['timestamp'] for entry in filtered_data], counts, label=id_to_name[station_id])
                
                if compare_with_soll:
                    plt.axhline(y=soll_value, color='r', linestyle='--', label=f'Soll-Wert {id_to_name[station_id]}')
    else:
        if station_id_requested in lubu_ids and station_id_requested in station_data['station_id'].values:
            filtered_data = station_data[station_data['station_id'] == station_id_requested].to_dict('records')
            
            if vehicle_type_name == 'all':
                counts = [entry['num_bikes_available'] for entry in filtered_data]
                soll_value = soll_werte[soll_werte['station_id'] == station_id_requested][['bike', 'pedelec', 'cargopedelec']].sum(axis=1).values[0]
            else:
                counts = extract_vehicle_count(filtered_data, vehicle_type_name)
                soll_value = soll_werte[soll_werte['station_id'] == station_id_requested][soll_column].values[0]
            
            plt.plot([entry['timestamp'] for entry in filtered_data], counts, label=id_to_name[station_id_requested])
            
            if compare_with_soll:
                plt.axhline(y=soll_value, color='r', linestyle='--', label=f'Soll-Wert {id_to_name[station_id_requested]}')
        else:
            print(f"Keine Daten für Station ID {station_id_requested} gefunden.")
            return
    
    plt.title('Verfügbare Fahrräder an Stationen über Zeit')
    plt.xlabel('Zeit')
    plt.ylabel('Verfügbare Fahrräder')
    plt.legend()
    plt.tight_layout()
    plt.show()

# Generierung von Stationsinformationen
def generate_station_info():
    data = {
        'station_id': lubu_ids,
        'name': [f'Station {i+1}' for i in range(len(lubu_ids))],
        'capacity': [20] * len(lubu_ids)
    }
    return pd.DataFrame(data)

# Interaktive Abfrage
station_id_input = input("Für welche Station möchten Sie den Ist Zustand analysieren? (Geben Sie die station_id ein oder 'alle' für alle Stationen): ")
bike_type = input("Möchten Sie nur eine spezielle Fahrradart analysieren? (all/bike/pedelec/cargopedelec): ").lower()
compare_with_soll = input("Möchten Sie einen Vergleich mit den Soll-Werten anzeigen? (Ja/Nein): ").lower() == 'ja'

visualize_station_data(station_id_input if station_id_input != "alle" else "alle", bike_type, compare_with_soll)
